# Importing Libraries

* System Append to set proper path

In [ ]:
sys.path.append('../')

* Default

In [ ]:
import lasio
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from collections import Counter
import time

* Pandas Options

In [ ]:
pd.set_option('max_columns', None)

* Source Code

In [ ]:
from Source.Utils import welllog
from Source.Utils import multi_df
from Source.Utils import well_plot

* Tqdm Progress Bar

In [ ]:
%%capture
from tqdm import tqdm_notebook

# Checkpoint import

In [ ]:
total_df = pd.read_csv('../checkpoints/litho_data_3.csv.gz', compression='gzip') # already non-null litho classes

In [ ]:
total_df.head()

In [ ]:
total_df['LITHOLOGY_GEOLINK_2'] = total_df['LITHOLOGY_GEOLINK']

for code in tqdm_notebook(total_df.index, desc='Regrouping lithologies'):

    if (total_df['LITHOLOGY_GEOLINK_2'][code] == 2 or total_df['LITHOLOGY_GEOLINK_2'][code] == 3):

        total_df['LITHOLOGY_GEOLINK_2'][code] = 1

    elif total_df['LITHOLOGY_GEOLINK_2'][code] == 6:

        total_df['LITHOLOGY_GEOLINK_2'][code] = 5

    elif total_df['LITHOLOGY_GEOLINK_2'][code] == 34:

        total_df['LITHOLOGY_GEOLINK_2'][code] = 33


    elif total_df['LITHOLOGY_GEOLINK_2'][code] == 13:

        total_df['LITHOLOGY_GEOLINK_2'][code] = 12

    else:
        
        pass

# Lithology Code Prediction

* LITHOLOGY_GEOLINK non NULL data

In [ ]:
#litho_data = total_df.drop(columns=['WELL_NAME', 'LITHOLOGY_GEOLINK_2', 'LITHOLOGY_GEOLINK_3', 'DRDN', 'AI']) # litho1 optimum set (84.9)

litho_data = total_df.drop(columns=['WELL_NAME', 'LITHOLOGY_GEOLINK', 'LITHOLOGY_GEOLINK_3', 'DRDN', 'AI']) # litho2 optimum set (84.5)

# litho_data = total_df.drop(columns=['WELL_NAME', 'LITHOLOGY_GEOLINK_2', 'LITHOLOGY_GEOLINK']) # litho3 optimum set

litho_data.head(n=5)

In [ ]:
len(litho_data)

    * Converting LITHOLOGY_GEOLINK to int type

In [ ]:
litho_data['LITHOLOGY_GEOLINK_2'] = litho_data['LITHOLOGY_GEOLINK_2'].astype(int)

    * Checking classes balance

In [ ]:
unsorted_bal_class = dict(Counter(litho_data['LITHOLOGY_GEOLINK_2'].values))

sorted_bal_class = {k: v for k, v in sorted(unsorted_bal_class.items(), key=lambda item: item[1])}

sorted_bal_class # although some classes have a considerable less representation than others, we need to respect this distribution to maintain the geological setting of the area

    * Pearson Correlation

In [ ]:
# corrmat_litho = abs(litho_data.corr()) # absolute correlation

# plt.figure(figsize=(25,15))

# sns.heatmap(corrmat_litho, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, cmap='Blues')

    * Dataset Creation

In [ ]:
from sklearn.model_selection import train_test_split

X = litho_data.drop(columns='LITHOLOGY_GEOLINK_2')

Y = litho_data['LITHOLOGY_GEOLINK_2'].values

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.5, random_state=42) # close to the tabnet training dataset size

In [ ]:
print('### Training Dataset: ### ', x_train.shape, y_train.shape)
print('### Test Dataset: ### ', x_test.shape, y_test.shape)

    * Model Selection

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn import linear_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV, cross_validate

class_pipelines = []

class_pipelines.append(('ScaledRidge', Pipeline([('Scaler', RobustScaler()),('Ridge', linear_model.RidgeClassifier())])))
class_pipelines.append(('ScaledSGDCls', Pipeline([('Scaler', RobustScaler()),('SGDCls', linear_model.SGDClassifier(n_jobs=30))])))
class_pipelines.append(('ScaledKNNCls', Pipeline([('Scaler', RobustScaler()),('KNNCls', KNeighborsClassifier(n_jobs=30))])))
class_pipelines.append(('ScaledDTC', Pipeline([('Scaler', RobustScaler()),('DTC', DecisionTreeClassifier())])))
class_pipelines.append(('ScaledRFC', Pipeline([('Scaler', RobustScaler()),('RFC', RandomForestClassifier(n_jobs=30))])))
class_pipelines.append(('ScaledADA', Pipeline([('Scaler', RobustScaler()),('ADA', AdaBoostClassifier())])))
class_pipelines.append(('ScaledGBC', Pipeline([('Scaler', RobustScaler()),('GBC', GradientBoostingClassifier())])))
class_pipelines.append(('ScaledXBG', Pipeline([('Scaler', RobustScaler()),('XBG', XGBClassifier(n_jobs=30))])))
class_pipelines.append(('ScaledCAT', Pipeline([('Scaler', RobustScaler()),('CAT', CatBoostClassifier())])))
class_pipelines.append(('ScaledLGBM', Pipeline([('Scaler', RobustScaler()),('LGBM', LGBMClassifier(n_jobs=30))])))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score, make_scorer

results = []

names = []

for name, model in tqdm_notebook(class_pipelines, desc='Cross-Validation Procedure'):

    kfold = KFold(n_splits=5, random_state=42)

    scorers = {'accuracy': make_scorer(accuracy_score), 'balanced_accuracy': make_scorer(balanced_accuracy_score), 'f1': make_scorer(f1_score, average='macro')}

    final_scorers = cross_validate(model, x_train, y_train, cv=kfold, scoring=scorers)
    results.append(final_scorers)
    names.append(name)
    print(name, 'Acc: ', final_scorers['test_accuracy'].mean(), '\\', final_scorers['test_accuracy'].std(), '\n')
    print(name, 'BalAcc: ', final_scorers['test_balanced_accuracy'].mean(), '\\', final_scorers['test_balanced_accuracy'].std(), '\n')
    print(name, 'F1: ', final_scorers['test_f1'].mean(), '\\', final_scorers['test_f1'].std(), '\n')


    * Hyperparameter Tunning

In [ ]:
# start_time = time.time()

# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# # Number of features to consider at every split
# max_features = ['auto', 'sqrt']
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]
# # Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rfc = RandomForestClassifier(n_jobs=60)

# kfold = KFold(n_splits=5, random_state=42)

# rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = kfold, verbose=10, random_state=42, scoring='balanced_accuracy')

# rfc_random.fit(RobustScaler().fit_transform(x_train), y_train)

# print(str(round((time.time() - start_time)/60,1)) + ' minutes taken') 


In [ ]:
# result_cv = pd.DataFrame.from_dict(rfc_random.cv_results_)

# result_cv.to_csv('../models/results/rand_search_rfc.csv')

In [ ]:
start_time = time.time()

best_rfc = RandomForestClassifier(max_depth=40, n_estimators=1400, max_features='auto', min_samples_split=2, bootstrap=False, n_jobs=60) # the rest of the best parameters are  the default ones

best_rfc.fit(RobustScaler().fit_transform(x_train), y_train)

print(str(round((time.time() - start_time)/60,1)) + ' minutes taken') 

    * Model Accuracy Test

In [ ]:
start_time = time.time()

y_predict = best_rfc.predict(RobustScaler().fit_transform(x_test))

print(str(round((time.time() - start_time)/60,1)) + ' minutes taken') 

In [ ]:
print('Accuracy: ', str(accuracy_score(y_test, y_predict)), '\n')
print('########################', '\n')
print('Balanced Accuracy: ', str(balanced_accuracy_score(y_test, y_predict)))
print('########################', '\n')
print('F1-Score Micro: ', str(f1_score(y_test, y_predict, average='weighted')))

In [ ]:
tree_feature_importances = (best_rfc.feature_importances_)
sorted_idx = tree_feature_importances.argsort()
y_ticks = np.arange(0, len(X.columns))
fig, ax = plt.subplots()
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(X.columns[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()

    * Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix

fig, ax = plt.subplots(figsize=(10, 10))

plot_confusion_matrix(best_rfc, RobustScaler().fit_transform(x_test), y_test, normalize='true', cmap=plt.cm.Blues, ax=ax, values_format='.1f')

plt.show()

In [ ]:
tree_feature_importances = best_rfc.feature_importances_
sorted_idx = tree_feature_importances.argsort()

y_ticks = np.arange(0, len(X.columns))
fig, ax = plt.subplots()
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(X.columns[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()

    * Null Values Prediction

        * Null Values Dataset 

null_litho_dataset = total_df[pd.isnull(total_df['LITHOLOGY_GEOLINK'])].drop(columns=['WELL_NAME', 'LITHOLOGY_GEOLINK'])

        * Prediction

litho_prediction = best_rfc.predict(RobustScaler().fit_transform(null_litho_dataset))

        * Replacing Null Values for Predicted Ones

predicted_litho_dataset = null_litho_dataset

predicted_litho_dataset['LITHOLOGY_GEOLINK'] = litho_prediction

predicted_litho_dataset.head(n=5)

    * Final Dataframe

final_df = litho_data.append(predicted_litho_dataset)

final_df.sort_index(inplace=True)

final_df['WELL_NAME'] = total_df['WELL_NAME'].values

final_df.head(n=5)

# Checkpoints

path_file_final_df = '../checkpoints/final_df.csv.gz'

final_df.to_csv(path_file_final_df,index=False, compression='gzip')